# Fusions des embbeddings pour CamemBERT

## Import

In [1]:
import pandas as pd
import numpy as np
from itertools import chain, combinations

from sklearn.model_selection import cross_val_score
import sklearn.model_selection
from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split

## Fonctions utilitaires (combinaisons, convsion liste/dictionnaire...)

In [41]:
# Transforme une liste de fichiers en dictionnaire de fichiers avec les même proportion d'embbeddings
def listToDicSameProp(lst):
    return {elt: 1./len(lst) for elt in lst}

# Créer toute les combinaisons
def listToDicCombiSameProp(lst, minLen=1, maxLen=-1):
    if maxLen == -1:
        maxLen = len(lst)
    return [listToDicSameProp(x) for x in chain.from_iterable(combinations(lst, r) for r in range(minLen, maxLen+1))]

# Fusion de listes
def listMerge2(lst1, lst2, prop1):
    return [{elt1: prop1, elt2: 1.-prop1} for elt1 in lst1 for elt2 in lst2]

# Effectu un dégradé sur le coefficient entre deux embbeddings
def eltsGradient(elt1, elt2):
    return [{elt1: x, elt2: 1.-x} for x in np.arange(.1, 1., 0.1)]

# Convertie une liste en dictionnaire avec les éléments de la
# liste lst comme clé et une valeur unique val comme valeur
def listToDic(lst, val):
    return {elt: val for elt in lst}

# Convertie plusieurs liste en plusieurs dictionnaires avec les éléments des
# listes lsts[i] comme clé et une valeur unique val comme valeur
def listsToDic(lsts, val):
    return [listToDic(lst, val) for lst in lsts]

# Renvoi une liste dans laquel chaque élément est une liste avec un élément de moins à chaque fois :
# [1, 2, 3] devient [[1], [1, 2], [1, 2, 3]]
def decrList(lst):
    return [lst[0:i+1] for i in range(len(lst))]

## Chargement d'un ensemble d'embbeddings, fusion et tests
Cette fonction charge un ensemble d'embbeddings et fait le mélange entre ces embbeddings :
- Soit avec une moyenne pondéré par les coeficients indiqué ($x_j = \sum_{i=0}^n c_i x_{ij}$ avec $x_j$ l'embbedding de la phrase $j$ résultant, $n$ le nombre de fichiers, $c_i$ le coefficient du fichier $i$ et $x_ij$ l'embbedding de la phrase $j$ du fichier $i$).
- Soit avec une concaténation des embbeddings (donc l'embbedding $x_j$ sera de dimension $\sum_{i=0}^n dim(x_ij)$)

Ensuite le SVM calcul le résultat sur ces nouveau embbeddings.

In [49]:
def testEmbd(filenames, concat=False):
    # Chargement des données
    X = np.ndarray((0,0))
    y = []
    for embdFile, proportion in filenames.items():
        dataFile = np.load(embdFile, allow_pickle=True)
        if concat:
            proportion = 1.
        if len(X) == 0:
            X = proportion*dataFile
        else:
            if concat:
                X = np.concatenate((X, proportion*dataFile), axis=1)
            else:
                X += proportion*dataFile
    y = np.load("label.npy")

    # Tests
    cv = sklearn.model_selection.ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
    clf = svm.SVC(kernel='rbf', C=1, random_state=42)
    results = cross_val_score(clf, X, y, cv=cv, scoring="f1")
    print("F-score de la validation croisée : ", results)
    print("Moyenne des F-score", np.mean(results))
    return np.mean(results)

## Fonction de tests de différentes combinaisons d'embbeddings

In [1]:
def batchTestEmbd(filenamesLists, concat=False):
    print(len(filenamesLists))
    return {str(filenames): testEmbd(filenames, concat) for filenames in filenamesLists}

In [51]:
# Liste des tests a effectuer
# Pour chaque test : un dictionnaire fichier embbedding -> ratio de ce fichier
filenamesLists = listsToDic(decrList(["POS5.npy", "LRInv1.npy", "POS3.npy", "POS4.npy", "POS1.npy", "POS2.npy", "alea3.npy", "LRInv3.npy", "LRInv2.npy", "LRInvTotal.npy", "alea1.npy", "alea4.npy", "alea5.npy", "alea2.npy", "messages_equilibre_ok.npy"]), 1.)
batchTestEmbd(filenamesLists, True)

[{'POS5.npy': 1.0},
 {'POS5.npy': 1.0, 'LRInv1.npy': 1.0},
 {'POS5.npy': 1.0, 'LRInv1.npy': 1.0, 'POS3.npy': 1.0},
 {'POS5.npy': 1.0, 'LRInv1.npy': 1.0, 'POS3.npy': 1.0, 'POS4.npy': 1.0},
 {'POS5.npy': 1.0,
  'LRInv1.npy': 1.0,
  'POS3.npy': 1.0,
  'POS4.npy': 1.0,
  'POS1.npy': 1.0},
 {'POS5.npy': 1.0,
  'LRInv1.npy': 1.0,
  'POS3.npy': 1.0,
  'POS4.npy': 1.0,
  'POS1.npy': 1.0,
  'POS2.npy': 1.0},
 {'POS5.npy': 1.0,
  'LRInv1.npy': 1.0,
  'POS3.npy': 1.0,
  'POS4.npy': 1.0,
  'POS1.npy': 1.0,
  'POS2.npy': 1.0,
  'alea3.npy': 1.0},
 {'POS5.npy': 1.0,
  'LRInv1.npy': 1.0,
  'POS3.npy': 1.0,
  'POS4.npy': 1.0,
  'POS1.npy': 1.0,
  'POS2.npy': 1.0,
  'alea3.npy': 1.0,
  'LRInv3.npy': 1.0},
 {'POS5.npy': 1.0,
  'LRInv1.npy': 1.0,
  'POS3.npy': 1.0,
  'POS4.npy': 1.0,
  'POS1.npy': 1.0,
  'POS2.npy': 1.0,
  'alea3.npy': 1.0,
  'LRInv3.npy': 1.0,
  'LRInv2.npy': 1.0},
 {'POS5.npy': 1.0,
  'LRInv1.npy': 1.0,
  'POS3.npy': 1.0,
  'POS4.npy': 1.0,
  'POS1.npy': 1.0,
  'POS2.npy': 1.0,
  'alea